In [1]:
# Imported necessary libraries

import re
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Bidirectional, Dropout, Embedding, SpatialDropout1D, LSTM
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import models

In [5]:
df = pd.read_csv("Tweets.csv")
df.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [6]:

print("Number of Rows: " + str(df.shape[0]))
print("Number of Columns: " + str(df.shape[1]))

Number of Rows: 14640
Number of Columns: 15


In [7]:

# Select Required Columns Only

tweet_df = df[['text','airline_sentiment']]

In [8]:
tweet_df.head()

,text,airline_sentiment
0,@VirginAmerica What @dhepburn said.,neutral
1,@VirginAmerica plus you've added commercials t...,positive
2,@VirginAmerica I didn't today... Must mean I n...,neutral
3,@VirginAmerica it's really aggressive to blast...,negative
4,@VirginAmerica and it's a really big bad thing...,negative


In [11]:
tweet_df = tweet_df[tweet_df['airline_sentiment'] != 'neutral']

In [12]:
# Attribute and Labels
X = tweet_df.text
y = tweet_df.airline_sentiment

In [13]:
# Train Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
# Checking Train Set

print("Training Set X Items: " + str(len(X_train)))
print("Training Set y Items: " + str(len(y_train)))

Training Set X Items: 9232
Training Set y Items: 9232


In [15]:
# Checking Test Set

print("Test Set X Items: " + str(len(X_test)))
print("Test Set y Items: " + str(len(y_test)))

Test Set X Items: 2309
Test Set y Items: 2309


In [16]:
# Getting required labels only and encoding

review_labels_train = y_train.factorize()

In [17]:
review_labels_train[0]

array([0, 0, 0, ..., 1, 0, 0], dtype=int64)

In [18]:
# Check Review Labels
review_labels_train[1]

Index(['negative', 'positive'], dtype='object')

In [19]:
vocab_size = X_train.values
vocab_size

array(['@USAirways Another dead end.  They only handle AA L&amp;F.  They gave me the same failed # I already had. 610-362-7498(99) VM full.  #lost',
       '@USAirways #2066. Was on plane from PBI to CLT and knew about the frozen water. Also saw a plane to NYC take off at the gate next door!',
       '@USAirways waiting for bags now over 25min in Phl bag claim!',
       ...,
       'Lovely! RT @JetBlue: Our fleet’s on fleek. http://t.co/Hi6Fl1AX9E',
       "@united Okay thanks if you could please update me. I was told at the airport someone would call me today but they haven't.",
       '@USAirways IS THIS RINGLING BROTHERS BARNUM AND BAILEY???  SHOULD I KEEP MY EYES PEELED FOR THE CLOWN CAR???'],
      dtype=object)

In [20]:
# Vectorize a text corpus, by turning each text into sequence of integers

tokenizer = Tokenizer(num_words=8000,oov_token='OOV')
tokenizer.fit_on_texts(vocab_size)

vocab_size = len(tokenizer.word_index) + 1

print(tokenizer)
print(vocab_size)

11636


In [21]:
len(tokenizer.word_index)

11635

In [22]:
# Getting required labels only and encoding

review_labels_train = y_train.factorize()

In [30]:
review_labels_train

(array([0, 0, 0, ..., 1, 0, 0], dtype=int64),
 Index(['negative', 'positive'], dtype='object'))

In [31]:
vocab_size = X_train.values
vocab_size

array(['@USAirways Another dead end.  They only handle AA L&amp;F.  They gave me the same failed # I already had. 610-362-7498(99) VM full.  #lost',
       '@USAirways #2066. Was on plane from PBI to CLT and knew about the frozen water. Also saw a plane to NYC take off at the gate next door!',
       '@USAirways waiting for bags now over 25min in Phl bag claim!',
       ...,
       'Lovely! RT @JetBlue: Our fleet’s on fleek. http://t.co/Hi6Fl1AX9E',
       "@united Okay thanks if you could please update me. I was told at the airport someone would call me today but they haven't.",
       '@USAirways IS THIS RINGLING BROTHERS BARNUM AND BAILEY???  SHOULD I KEEP MY EYES PEELED FOR THE CLOWN CAR???'],
      dtype=object)

In [35]:
# Vectorize a text corpus, by turning each text into sequence of integers

tokenizer = Tokenizer(num_words=8000,oov_token='OOV')
tokenizer.fit_on_texts(vocab_size)

vocab_size = len(tokenizer.word_index) + 1

print(tokenizer)
print(vocab_size)

11636


In [36]:
# Store and Padding Converted Sequences
tweet = X_train.values

tweet_seqs = tokenizer.texts_to_sequences(tweet)

padded_sequence_train = pad_sequences(tweet_seqs, maxlen=200)
print(padded_sequence_train)

[[   0    0    0 ... 3560  364  141]
 [   0    0    0 ...   76  196  754]
 [   0    0    0 ...  304   79  354]
 ...
 [   0    0    0 ...   48   50    1]
 [   0    0    0 ...   32   57  353]
 [   0    0    0 ...    3 4197  569]]


In [37]:
print(padded_sequence_train[0])

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0   13  143 1590  549   57  122  660  221 2105
   59  752   57  428   20    3  257  855    4  224   81 5034 3559 5035
 2830 

In [38]:
embedding_vector_length = 32

model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_length,     
                                     input_length=200) )
model.add(SpatialDropout1D(0.25))
model.add(LSTM(50, dropout=0.5, recurrent_dropout=0.5))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 200, 32)           372352    
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 200, 32)           0         
_________________________________________________________________
lstm (LSTM)                  (None, 50)                16600     
_________________________________________________________________
dropout (Dropout)            (None, 50)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 51        
Total params: 389,003
Trainable params: 389,003
Non-trainable params: 0
_________________________________________________________________
None


In [39]:
trained = model.fit(padded_sequence_train,review_labels_train[0], validation_split=0.2, epochs=5, batch_size=32)

Epoch 1/5
231/231 [==============================] - 60s 245ms/step - loss: 0.4185 - accuracy: 0.8246 - val_loss: 0.3123 - val_accuracy: 0.8717
Epoch 2/5
231/231 [==============================] - 56s 241ms/step - loss: 0.2308 - accuracy: 0.9128 - val_loss: 0.2397 - val_accuracy: 0.8993
Epoch 3/5
231/231 [==============================] - 52s 227ms/step - loss: 0.1555 - accuracy: 0.9439 - val_loss: 0.2580 - val_accuracy: 0.9009
Epoch 4/5
231/231 [==============================] - 59s 254ms/step - loss: 0.1165 - accuracy: 0.9584 - val_loss: 0.2359 - val_accuracy: 0.9015
Epoch 5/5
231/231 [==============================] - 57s 249ms/step - loss: 0.0990 - accuracy: 0.9644 - val_loss: 0.2604 - val_accuracy: 0.9096
